In [2]:
from PIL import Image
import sys
import pathlib

file = pathlib.Path("build/12-30.png")

# Image to hold the quantize palette
pal_img = Image.new("P", (1, 1))

# Image *should* already be using just these colours
pal_img.putpalette((
    0, 0, 0,
    255, 255, 255,
    0, 255, 0,
    0, 0, 255,
    255, 0, 0,
    255, 255, 0,
    255, 128, 0,
    # 220, 180, 200
))

# Open the source image and quantize it to our palette
i = Image.open(file).convert("RGB").quantize(palette=pal_img)

WIDTH, HEIGHT = 800, 480

i = i.crop((0, 0, WIDTH, HEIGHT))

# save image back out for debugging
i.save(file.with_suffix(".inky.png"))

buf = bytearray(WIDTH * HEIGHT // 8 * 3)

def put_pixel(x, y, c):
    offset = (WIDTH * HEIGHT) // 8
    pixel = (x // 8) + (y * WIDTH // 8)
    
    bit_offset = 7 - (x & 0b111)
    
    b = (c & 0b100) >> 2

    buf[pixel] &= ~(1 << bit_offset)
    buf[pixel] |= (b << bit_offset)

    pixel += offset

    b = (c & 0b010) >> 1
    buf[pixel] &= ~(1 << bit_offset)
    buf[pixel] |= (b << bit_offset)

    pixel += offset
    
    b = (c & 0b001)
    buf[pixel] &= ~(1 << bit_offset)
    buf[pixel] |= (b << bit_offset)

# Copy the source image pixels into our buffer
for x in range(WIDTH):
    for y in range(HEIGHT):
        put_pixel(x, y, i.getpixel((x, y)))

# Write our bin file
with open(file.with_suffix(".inky.bin"), "wb+") as f:
    f.write(buf)